#  Submit GW for molecules

In [1]:
# General imports.
import nglview
import numpy as np
import ipywidgets as ipw
from IPython.display import clear_output
import itertools

from traitlets import dlink

# AiiDA imports.
%aiida
from aiida.orm import Dict as Dict
from aiida.orm import SinglefileData, Node

# Workchains.
from aiida.plugins import WorkflowFactory



# AiiDA lab imports.
from aiidalab_widgets_base.utils import string_range_to_list, list_to_string_range
from aiidalab_widgets_base import CodeDropdown, StructureManagerWidget, BasicStructureEditor
from aiidalab_widgets_base import StructureBrowserWidget, StructureUploadWidget, SubmitButtonWidget, SmilesWidget

# Custom imports.
from apps.surfaces.widgets.create_xyz_input_files import make_geom_file
from apps.surfaces.widgets.ANALYZE_structure import StructureAnalyzer
from apps.surfaces.widgets.build_slab import BuildSlab
from apps.surfaces.widgets.cp2k2dict import CP2K2DICT
from apps.surfaces.widgets.inputs import InputDetails
from apps.surfaces.widgets.empa_viewer import EmpaStructureViewer
from apps.surfaces.widgets.import_cdxml import CdxmlUpload2GnrWidget

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
Cp2kAdsorbedGwIcWorkChain = WorkflowFactory('nanotech_empa.cp2k.ads_gw_ic')

In [4]:
# Structure selector.
empa_viewer = EmpaStructureViewer()

structure_selector = StructureManagerWidget(
    viewer=empa_viewer,
    importers=[
        StructureBrowserWidget(title="AiiDA database"),
        StructureUploadWidget(title="Import from computer"),
        SmilesWidget(title="From SMILES"),
        CdxmlUpload2GnrWidget(title="CDXML"),
    ],
    editors = [
        BasicStructureEditor(title="Edit structure")
    ],
    storable=False, node_class='StructureData')
display(structure_selector)

# Code.
computer_code_dropdown = CodeDropdown(input_plugin='cp2k')
#input_details = InputDetails()
create_input = ipw.Button(description="Create Input")

StructureManagerWidget(children=(Tab(children=(StructureBrowserWidget(children=(VBox(children=(VBox(children=(…

/opt/conda/lib/python3.7/site-packages/aiida/orm/computers.py:757: AiidaDeprecationWarning: this property is deprecated, use the `label` property instead
  warnings.warn('this property is deprecated, use the `label` property instead', AiidaDeprecationWarning)  # pylint: disable=no-member


In [ ]:
#dlink((empa_viewer, 'details'), (input_details, 'details'))
#dlink((computer_code_dropdown, 'selected_code'),(input_details, 'selected_code'))

def get_builder_gw():
    
    builder = Cp2kAdsorbedGwIcWorkChain.get_builder()

    builder.metadata.description = 'a description'
    builder.code = computer_code_dropdown.selected_code

    slab_included = True

#   builder.ads_height = Float(3.0) no slab

    ase_geom =  structure_selector.structure        

    # Convert ase tags to magnetization list
    mag_list = [
        -1 if t == 1 else 1 if t == 2 else 0 for t in ase_geom.get_tags()
    ]  
    builder.structure = structure_selector.structure_node # StructureData(ase=ase_geom)
    builder.magnetization_per_site = List(list=mag_list)

    builder.protocol = Str('gpw_std')
    builder.multiplicity = Int(1)

    builder.geometry_mode = Str('ads_geo')

    builder.debug = Bool(True)
    builder.walltime_seconds = Int(5 * 60)
    builder.max_nodes = Int(1)    
    

    return builder



In [ ]:
def after_submission(_=None):   
    structure_selector.value = None


btn_submit_gw = SubmitButtonWidget(Cp2kAdsorbedGwIcWorkChain, 
                                input_dictionary_function=get_builder_gw
                               )

btn_submit_gw.on_submitted(after_submission)

In [ ]:
output = ipw.Output()
def update_all(_=None):
    btn_submit.btn_submit.disabled=True    

def create(_=None):
    if computer_code_dropdown.selected_code is None:
        msg = 'please select a computer'
        can_submit = False
    else:            
        can_submit, msg, details_dict = input_details.return_final_dictionary()
       # if len(details_dict['description']) < 4 :
       #     msg = 'Please provide a better description'
       #     can_submit = False
            
        btn_submit.btn_submit.disabled = not can_submit
    with output:
        clear_output()
        if can_submit:
            print("Ready to submit a ",details_dict['workchain']," type of calculation" )
            if details_dict['workchain'] == 'GWWorkChain':
                display(btn_submit_gw)
            else:
                display(btn_submit)
        else:
            print(msg)


In [ ]:
display(ipw.VBox([computer_code_dropdown,btn_submit_gw]), output)